$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

**Answer:**   
The receptive field is the area in the input space which produces the features, the receptive fields of different features partially overlap and as such cover the entire input space. 
When stacking convolutional layers, the receptive fields combine and each feature takes input from a larger area of pixels in the previous layer image. When the receptive field grows, while progressing through the convolutional layers, it takes into account the value of a pixel and it's surrounding pixels (the closer to the center a pixel is, the higher it's value is).
    

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

**Answer:**  
Three different ways to control the rate at which the receptive field grows:


1. Pooling:  
The pooling layer is a way to reduce the dimension of the feature map, by combining features in the same region. Thus, the successive conv layers are affected by increasingly larger parts of the input image, which results in a rapid increase in the receptive field size.  

2. Stride:  
Stride determines the shift size of the filter, in other words it detrmines how big is the overlapping portion of the receptive fields between features. Therefore, the larger the stride is, the smaller the overlapping portion of pixels are between features, the larger the receptive filed will grow between layers.

3. Dilation:   
Dilation determines the spacing between pixels in the filter frame. Using delation grows the receptive field exponentialy between layers without effecting the paramteres, which will continue to grow linearly. 

Pooling combines feaures in the same region, which makes the model more robust to variations in the position of the input features. Unlike Stride, which in each conv layer combines the features while producing the feature map. Finally, with Dilation the weights are matched to distant pixels in the input image, the distance in determind by delation parameter, and so this results in a more globel context of the input.



3. Imagine a CNN with three convolutional layers, defined as follows:

In [1]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

**Answer:**  
Each receptive field size is derived from the layer before it, hence we can calculate the network receptive field recursively. For convenience let's look at the relevant data for each layer in the table below:  

<table style="width:60%">
    <tr>
        <th></th>
        <th> layer 1 </th>
        <th> layer 2 </th>
        <th> layer 3 </th>
        <th> layer 4 </th>
        <th> layer 5 </th>
    </tr>
    <tr>
        <th> k </th>
        <td> 3 </td>
        <td> 2 </td>
        <td> 5 </td>
        <td> 2 </td>
        <td> 7 </td>
    </tr>
    <tr>
        <th> s </th>
        <td> 1 </td>
        <td> 2 </td>
        <td> 2, 4 </td>
        <td> 2, 8 </td>
        <td> 1, 8 </td>
    </tr>
    <tr>
        <th> d </th>
        <td> 1 </td>
        <td> 1 </td>
        <td> 1 </td>
        <td> 1 </td>
        <td> 2 </td>
    </tr>
</table>

k = kernel, s = stride (layer stride, cumulative stride), d = diliation
Layer 2 and 4 are the max pooling layers, we'll treat them as if they were conv layer with k = 2 and s = 2. 

The recursive formula for the receptive field size of the output tensor: $ r_0 = \sum_{i=1}^L (d_i(k_i - 1) \cdot \Pi_{j=0}^{j-1} s_j) + 1 $

In the $ i $ iteration we calculate the receptive field of the $ i $ layer, which comes to be the sum of the previous layers receptive fields and the accumaltive stride. 
For example the first layer, the receptive field will be 3, since it's only affected by the kernel size. The formula produces the same result: $ r_1 = 1 \cdot (k_1 - 1) + 1 = (3 - 1) + 1 = 3 $.   
Now lets caclulate the outputs receptive field: 
$ r_0 = (1 \cdot (3 - 1) \cdot 1) +  
(1 \cdot (2 - 1) \cdot 1) + 
(1 \cdot (5 - 1) \cdot 1 \cdot 2) +
(1 \cdot (2 - 1) \cdot 1 \cdot 2 \cdot 2) + 
(2 \cdot (7 - 1) \cdot 1 \cdot 2 \cdot 2 \cdot 2) + 1 = 112 $

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

**Answer:**  
The layers before this change were learning the true output - $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$.
By changing to residual mapping, the residual block now learns the "residual mapping" - $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$. If we rearange this expression we get - $ f_l(\vec{x};\vec{\theta}_l) = \vec{y}_l - \vec{x} $, there is a an identity connection due to x, therefore the layers are trying to learn the delta. As a result the network produces different filters.  


### Dropout

1. Consider the following neural network:

In [ ]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

**Answer:**  
The Dropout layer with parameter p will 'zero' an element with a probability of p, or keep the element with a probability of $(1-p)$.  
In order to combine the dropout layers, will examine the probability for element X to be dropped out. The probability to drop the X element in the first dropout layer is $ p_1 $. The probability to drop the X element in the second dropout layer, is the probability not to drop it in the first layer- $ (1 - p_1) $ multiplyed by the probability to drop it in the second layer $ p_2 $.   
Finally we get: $ q = p_1 + (1 - p_1) \cdot p_2 $


2. **True or false**: dropout must be placed only after the activation function.

**Answer:**   
False.
Common parctice is to apply Dropout after non-linear activation functions. However, when using rectified linear units, such as ReLU, the decision for placing the Dropout layer before or after the activation layer will depend on the perticular code implementation due to computational efficiency. 
Let's demonstrate that for simple examples there is no difference:

Network A: fully connected, linear activation -> ReLU -> Dropout -> ...  
Network B: fully connected, linear activation -> Dropout -> ReLU -> ...

Let the output of the linear activation be: $[-6, -7, -8, 9, 1, 2]$.  
For both networks will use Dropout with $p = 0.5$, and will assume that the elements that are being zeroed are 2, 4, 6.  

Let us pass it through A first:  
ReLU($[-6, -7, -8, 9, 1, 2]$) = $[0, 0, 0, 9, 1, 2]$   
Dropout($[0, 0, 0, 9, 1, 2]$) = $[2*0, 0, 2*0, 0, 2*1, 0] = [0, 0, 0, 0, 2, 0] $

Now through B:   
Dropout($[-6, -7, -8, 9, 1, 2]$) =$[2*(-6), 0, 2*(-8), 0, 2*1, 0] = [-12, 0, -16, 0, 2, 0]$   
ReLU($[-12, 0, -16, 0, 2, 0]$) = $[0, 0, 0, 0, 2, 0]$  

As excpected, we got the same output from both networks.

3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

**Answer:**  
Dropout with parameter $p$ will drop the input elements with $p$ probability, and keep the elements with probability $(1-p)$. 
Therefore, the elements who 'survive' the dropout are multiplied by $(1-p)$ (as we saw in the example above). In order to maintain the value of each activation unchanged we need to undo the multipcation added by the dropout. Therefore we must scale by $1/ (1-p)$ to undo this. 

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

**Answer:**  
We would not train a binary classification model with the L2 loss function (MSE), a more fitting loss function is the Cross-Entropy loss function.  
The reason for this is that the MSE function expects real-value inputs in range $(- \infty, \infty)$, while binary classification modles output probabilities in 
range $(0, 1)$ through the sigmoid/ logistic function.   
Let's demonstarte this idea with a graph: 

<center><img src="losses q1.png" width="1000" height="1000" /></center>

The graph displays the MSE (blue) with the sigmoid activation, to normalize the output to fit the range $(0,1)$, and the cross-entropy (red) also with the sigmoid activation.  
It can be seen in the graph, that for example when the output is $-10$, the MSE has no slope, therefore the gradient will get stuck, and the model or will train very slowly or won't be able to train. On the other hand the cross entropy has a slope and therefore the model will be able to train.   
Direct calculation of the gradient produces the following result: 

$$ L_2 = (1- {{1} \over {1 + e^-x}})^2 \Longrightarrow {{\partial L_2} \over {\partial \mat{x}}} = {{\mat{e}^{-\mat{x}} log(\mat{e})} \over {(1 + \mat{e}^{-\mat{x}})^2}} $$

For $\ \mat{x} = -10 \ $ the dervitive value is:    $ \ \ {{\mat{e}^{10} log(\mat{e})} \over {(1 + \mat{e}^{10})^2}} = 1.971e-5 \sim 0 $  
The result is as expected from the graph, numericaly this result is equivelnt to zero, and that is way $L_2$ isn't recommended for a binary classification model.


2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [ ]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

**Answer:**  
The model that is described above is very deep, and is only using sigmoid as an activation function. The `sigmoid` function has a single output bounded by $[0.0, 1.0]$, that saturates when its input is extremely negative or extremely positive. Therefore, it's most likly the model is suffering from a vanishing gradient, which causes the saturation in the `sigmoid` function and thus, seen as a plateau.

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

**Answer:**  
Replacing the `sigmoid` activations with `tanh` probably wont't help, because tanh suffers from vanishing gradient as well. The `tanh` and `sigmoid` activations are very similar, the difference between them is range, `tanh`'s range is $[-1.0, 1.0]$ as opposed to `sigmoid`'s range, which is $[0.0, 1.0]$. Therefore, if the model reached a plateau with `sigmoid` it's most likely to behave the same with `tanh`. It seems that in this case, the best course of action would be to go with ReLU activation. 

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
  1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
  1. The gradient of ReLU is linear with its input when the input is positive.
  1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

**Anaswer:**  
A. **False**. ReLU suffers less from vanishing gradient, but it does saturate when the input is extremely negative. 

B. **False**. The ReLU activation function is defined as $f(x) = max\{0, x\}$. If $x$, the input, is positive then $f(x) = x$, and we get the identity function.   
The gradient of the identity function is 1. Therefore, the gradient is not linear with respect to the input.  

C. **True**. There is a case when large weight updates can mean that the summed input to the activation function is always negative, regardless of the input to the network.  
It is likely that this happens when learning a large negative bias term for its weights.   

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

**Answer:**
*  GD - 
    * Uses the entire data set to calculate the gradient.
    * Slow and takes a long time to train.
    * Usually used on small data sets.
* SGD -
    * Calculates the Gradient using only one random sample from the data set.
    * Converges fast and used mostly on large data sets where the parameters update more frequently.
    * We can't use a vectorized approach to speed up the process.
* mini-batch SGD - 
    * Mixture of GD and SGD, uses mini-batch of samples from the data set.
    * Will converge fast like SGD.
    * We can use the vectorized approach on each mini-batch to speed up the process.


2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

**Answer:**  
2.1 SGD calculates the gradient of exactly one random sample from the data set and updates the parameters according to the sample gradient when GD calculates the gradient of the entire data set which will take more computation time and slows down the learning process so as a result SGD will converge a lot faster. 
On top of that SGD will result in a less over-fitted model than GD because in each iteration the parameters are updated according to one sample.

2.2 GD can't be used when the loss surface has lots of local minima, GD will then get stuck in one of them and probably will not learn past that point. If the loss surface is non-convex the GD will learn very slow and can be not useful as well.

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

**Answer:**  
It depends, Increasing the batch size in mini-batch SGD will bring the algorithm closer to GD so that the calculated gradient will be close to the minima so the number of iteration will increase because the steps the algorithm take towards convergence will be smaller, but because we now have GPU with twice the RAM we can now use vectorization with a bigger batch in each iteration so the __Time__ it will take to get to that loss value will decrease.

1. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

**Answer:**  
1. True, in SGD algorithm for each epoch for each sample we, calculate its gradient and update the weights respectivly.
1. False, SGD calculates gradients based on a single sample which will results in an error from the true gradient, so the learning graph can be very noisy and slower then GD but in practice because the time it takes to calculate a single sample gradient instead of the whole dataset is much smaller SGD convergence faster then GD when the data set is large enghoth  
1. True, because the SGD uses only one sample each time his step are relativley large and noisy compare to the GD step so even if SGD got to a local minima the next step will probably release him.
1. False, SGD calculates one samples gradient each time in oppose to GD that calculates the entire data set gradinet.
1. False, there is no way to garenty that SGD or GD will converge to a global minima, there are ways to increase the chances of that to happend but they both might miss it and got stuck on a local minima.
1. False, Assuming we have a loss surface with narrow ravine, we can approximate the loss surface to polynomial function, when doing so Newton's method will converge in  asingle iteration because it considers both gradient and curvature when for SGD with mumentum the convergence will happend after several iterations.

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

**Answer:**  
False, the outer level net dosn't depend on the inner optimization problem therfore the output of the inner net can be calculate using any method that will result in a good output.



6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
  1. Explain the concepts of "vanishing gradients", and "exploding gradients".
  2. How can each of these problems be caused by increased depth?
  3. Provide a numerical example demonstrating each.
  4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

**Answer:**  
1. vanishing gradients - when using gradient-based algorithms and backpropagation where each weight in the network updates according to the partial derivative of the error function concerning the current weight. In some cases, the gradient will decrease slowly which will result in a very slow learning rate as the depth of the net grows

exploding gradient - The same as vanishing gradients only when some activation function has a large gradient. In that case, the learning rate will be very large and can cause the model to miss its minima and keep growing and growing and ruin the learning process.

2. When the number of layers of the network grows the gradients accumulate through the backpropagation process so for deep layers the change needed to be made to each weight is very small or very large depending on the gradient which can result in vanishing or exploding gradients.

3. vanishing gradient - For example, the Hyperbolic tangent has a gradient in range (0,1), backpropagation computes gradients by the chain rule so for a net with *n* layers the accumulation will have the effect of multiplying those small gradients *n* times which will decrease them exponentially and can result in vanishing gradients for the early layers.

exploding gradient - let's assume we have an *n* layers net where the initial weights are larger than 1, so because of the backpropagation the gradients of the loss will be multiplied by those weights and will get exponentially larger as we dive deeper into the net which will result in an exploding gradient.

4. By looking at the loss graph through the learning rate we can determine which of the two problems we have, if the loss is increasing for a few iterations and keeps growing we can assume that we have exploding gradient because the model is not learning and the weights keep getting bigger and far from the ground truth.
on the other hand, if we see that the loss value stays fixed or decreasing very slowly we can assume that we have a vanishing gradient problem.

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y,\hat{y}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

**Answer:**  
We'll find the derevitives using the cahin rule as we saw in the course lectures.

For convenience: $ \mat{z} = \mat{W}_1 x+ b_1 $

$$ {{\partial \ell} \over {\partial \hat{y}}} = {{-y} \over \hat{y}} + {{1-y} \over {1 -\hat{y}}} = {{\hat{y} - \mat{y}} \over {\hat{y}(1- \hat{y})}} $$

$$ 
{{\partial L_s} \over {\partial \mat{W}_1}} = 
{{\partial \ell} \over {\partial \hat{y}}} * 
{{\partial \hat{y}} \over {\partial \mat{z}}} *
{{\partial \mat{z}} \over {\partial \mat{W}_1}} +
\lambda  \norm{\mat{W}_1}_F = 
{1 \over N }\sum_{i=1}^N{{\hat{y} - \mat{y}} \over {\hat{y}(1- \hat{y})}} \cdot
\mat{W}_2^T \cdot \varphi ' (\mat{z}) \cdot \mat{x}^T + \lambda  \norm{\mat{W}_1} _F
$$

$$
{{\partial L_s} \over {\partial \mat{W}_2}} = 
{{\partial \ell} \over {\partial \hat{y}}} * 
{{\partial \hat{y}} \over {\partial \mat{W}_2}} +
\lambda  \norm{\mat{W}_2}_F = 
{1 \over N }\sum_{i=1}^N{{\hat{y} - \mat{y}} \over {\hat{y}(1- \hat{y})}} \cdot
\varphi(\mat{z})^T + \lambda  \norm{\mat{W}_2}_F
$$

$$
{{\partial L_s} \over {\partial b_1}} = 
{{\partial \ell} \over {\partial \hat{y}}} * 
{{\partial \hat{y}} \over {\partial \mat{z}}} *
{{\partial \mat{z}} \over {\partial b_1}} =
{1 \over N }\sum_{i=1}^N{{\hat{y} - \mat{y}} \over {\hat{y}(1- \hat{y})}} \cdot
\mat{W}_2^T \cdot \varphi '(\mat{z})
$$

$$
{{\partial L_s} \over {\partial b_2}} =
{{\partial \ell} \over {\partial \hat{y}}} * 
{{\partial \hat{y}} \over {\partial b_2}} = 
{1 \over N }\sum_{i=1}^N{{\hat{y} - \mat{y}} \over {\hat{y}(1- \hat{y})}}
$$

$$
{{\partial L_s} \over {\partial \mat{x}}} =
{{\partial \ell} \over {\partial \hat{y}}} * 
{{\partial \hat{y}} \over {\partial \mat{z}}} *
{{\partial \mat{z}} \over {\partial \mat{x}}} = 
{1 \over N }\sum_{i=1}^N{{\hat{y} - \mat{y}} \over {\hat{y}(1- \hat{y})}} \cdot
\mat{W}_2^T \cdot \varphi ' (\mat{z}) \cdot \mat{W}_1^T
$$


2. Derivative a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
  1. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
  
  2. What are the drawbacks of this approach? List at least two drawbacks compared to AD.

**Answer:**  

A. Given that our loss function is $\mat{f}(\mat{x}) = \mat{W} \mat{x} + b$, by chosing a very small $\Delta$, we could use the formula above with our loss function, during backpropgation to update our weights.  
To get a more accurate approximation, we could select a small $\Delta$ from each sides of the input. By doing so we limit from both sides the results of the formula, and thus, update our wehights more accuratly.


B. Two drawbacks:   

Numerical: Using the dervitive formula will give us an approximation of the result since we are using a delta, as opposed to AD which gives as an exact result. 
    
Complexity: If we would have used this formula insead of AD, we would have to calculate $\mat{f}(\mat{x})$ twice, once with $\mat{x}_0$ and the second time with $\mat{x}_0 + \Delta \mat{x}$.  
Doing so is computionaly expensive, and depends on the number of parameters in $\mat{f}$.


3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [4]:
import torch

N, d = 100, 5
dtype = torch.float64
X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

# TODO: Calculate gradients numerically for W and b
grad_W = torch.zeros(W.shape, dtype=dtype)
grad_b = torch.zeros(b.shape, dtype=dtype)
delta = 1e-7
W_delta = torch.clone(W)
b_delta = torch.clone(b)

for i in range(d):
    for j in range(d):
        W_delta[i][j] = W[i][j] + delta
        loss_delta = foo(W_delta, b) 
        grad_W[i][j] = (loss_delta - loss) / delta
        W_delta[i][j] = W[i][j]

for i in range(d):
    b_delta[i] = b[i] + delta
    loss_delta = foo(W, b_delta)
    grad_b[i] = (loss_delta - loss) / delta
    b_delta[i] = b[i]

# TODO: Compare with autograd using torch.allclose()
loss.backward()
autograd_W = W.grad
autograd_b = b.grad

assert torch.allclose(grad_W, autograd_W)
assert torch.allclose(grad_b, autograd_b)

loss=tensor(1.4910, dtype=torch.float64, grad_fn=<MeanBackward0>)


### Sequence models

1. Regarding word embeddings:
  1. Explain this term and why it's used in the context of a language model.
  1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding? If yes, what would be the consequence for the trained model? if no, why not?

**Answer:**  
A. Word embedding in the context of language models is the representation of words as vectors that encode the meaning of the word, so that words that are closer in the vecotr space are expected to be closer in meaning. The mapping between words that are in a space with many dimensions per word to a continuous vector space with a much lower dimension, is done by neural networks and therefore by a language model.   

B. Yes, a language model could train with one-hot encoding instead of word embedding. But, there a few drawbacks to this method: 
 - We loose the semantics realationship between the words in the text. In this method, each vectorization is an orthogonal representation in another dimension. This could possibly result in low preformance of the model. 
 - Another disadvantage is the scale when the number of output labels is large. In language modeling the number of out put lables equals to the vacabulary size, this means that each iput feature (word) will be represented as a large vector, and most of the values in the vector will be zero.

2. Considering the following snippet, explain:
  1. What does `Y` contain? why this output shape?
  2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [8]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


**Answer:**  
A. Embedding is the mapping between words in a high dimension space to vectors in a (much) lower dimension space. Y contains the embedding of the word set X, the words in X are of shape $[5, 6, 7, 8]$. 
The embedding_dim represents the vector size, and so, Y is a tensor that contains the mapping between words of shape $[5, 6, 7, 8]$ to vectors of length 42000, that is why Y's shape is $[5, 6, 7, 8, 42000]$.

B. An impelmentation of Embedding using only tensors:   
Initialize a tensor of shape [num_embeddings, embedding_dim] with random values, which will be the dictionary. Each word in the input, will be used as the key to locate the correspanding row in the dictionary. The rows values will be the vector representing the word. The mapping described will produce the same embedding for the same word, and the final shape of the embedding will be [the words shape, embedding_dim].


3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
  1. TBPTT uses a modified version of the backpropagation algorithm.
  2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
  3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.

**Answer:**  
A. False. TBPTT uses the backpropasgation algoritham as is. The algoritham is performed for a fixed number of timesteps, which truncates the amount of calculations and the amount of RNN outputs saved to the memory (to solve the vanishing\ exploding weights). 

B.  Fasle. TBPTT limits the number of timestamps is accuumilated in the model before preforming the backpropgation algoritham. The length of the sequence doesn't play a role in this case, instead we could limit the number of timestamps to S. If we would limit the sequences length, we wouldn't address the problems TBPTT is trying to solve, since an input sequences could still be comprised of thousands of timesteps, then this will be the number of derivatives required for a single weight update. 

C.  True. In TBPTT the compuitaitonal graph is truncated every S timestamps. The inputs in that time frame are backpropgated on, and there is no memory of input that came before it (and obviouslly after). For input with a timestamp t, the farthest back relationship it can learn is of (S-t). Therefore, the model can learn relations between input that are at most S timesteps apart.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
  1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


**Answer:**  
A. The attention mechanism retains and utilises all the hidden states of the input sequence during the decoding process. It does this by creating a unique mapping between each time step of the decoder output to all the encoder hidden states. This means that for each output that the decoder makes, it has access to the entire input sequence and can selectively pick out specific elements from that sequence to produce the output.  
On the other hand, the standard seq2seq model uses only the last hidden state of the encoder RNN as the context vector for the decoder, and generally is unable to accurately process long input sequences.

B. Self-attention allows the model to learn to access information from the past hidden layer, at the cost of very expensive computational calculations in the decoder. 
A self-attention layer connects all positions with a constant number of sequentially
executed operations, whereas a recurrent layer requires O(n) sequential operations. To improve computational performance for tasks involving
very long sequences, self-attention could be restricted to considering only a neighborhood of size r. This answer is influenced by the paper "Attention Is All You Need".



### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

  1. Images reconstructed by the model during training ($x\to z \to x'$)?
  1. Images generated by the model ($z \to x'$)?

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
  1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
  2. Every time we feed an image to the encoder, then decode the result, we'll get the same reconstruction.
  3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
  1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
  2. It's crucial to backpropagate into the generator when training the discriminator.
  3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
  4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
  5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
  1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
  1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

**Answer:**  


2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?